In [ ]:
# now we implement a model that will predict based on the metadata given in the original data. 

!pip install xgboost
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import keras
import cv2 # computer vision library¨
import tensorflow as tf # machine learning library
import matplotlib.pyplot as plt # data visualization tool
from tensorflow.python.keras import backend as K #to utilize more of keras' functionality

In [ ]:
train = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
test = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')
sub = pd.read_csv('../input/siim-isic-melanoma-classification/sample_submission.csv')
train.head()

In [ ]:
train['sex'] = train['sex'].astype("category").cat.codes +1
train['anatom_site_general_challenge'] = train['anatom_site_general_challenge'].astype("category").cat.codes +1
test['sex'] = test['sex'].astype("category").cat.codes +1
test['anatom_site_general_challenge'] = test['anatom_site_general_challenge'].astype("category").cat.codes +1
train.head()

In [ ]:
 train['sex'] = train['sex'].fillna('male')
train['age_approx'] = train['age_approx'].fillna(train['age_approx'].mean())
train['anatom_site_general_challenge'] = train['anatom_site_general_challenge'].fillna('torso')
test['sex'] = test['sex'].fillna('male')
test['age_approx'] = test['age_approx'].fillna(train['age_approx'].mean())
test['anatom_site_general_challenge'] = test['anatom_site_general_challenge'].fillna('torso')
train.head()

In [ ]:
#to have a more balanced dataset, we create a new dataframe that contains a more equal percentage of each type of target image
df_0=train[train['target']==0].sample(600)
df_1=train[train['target']==1]
train=pd.concat([df_0,df_1])
train=train.reset_index()

In [ ]:
from sklearn.model_selection import train_test_split

x_train_df = train[['sex', 'age_approx','anatom_site_general_challenge']]
y_train_df = train['target']


x_train, x_val, y_train, y_val = train_test_split(x_train_df, y_train_df, test_size=0.2, random_state=42)


x_test = test[['sex', 'age_approx','anatom_site_general_challenge']]


train_DMatrix = xgb.DMatrix(x_train, label= y_train)
test_DMatrix = xgb.DMatrix(x_test)

In [ ]:
clf = xgb.XGBClassifier(n_estimators=2000, 
                        max_depth=10, 
                        objective='multi:softprob',
                        seed=0,  
                        #nthread=-1, 
                        learning_rate=0.15, 
                        num_class = 2, 
                        scale_pos_weight = (32542/584))

In [ ]:
clf.fit(x_train, y_train)


In [ ]:
preds = clf.predict_proba(x_val)[:,1]
target = [round(value) for value in preds]

In [ ]:
# evaluate predictions
accuracy = accuracy_score(y_val, target)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
#the submission file:
test_df = test[['sex', 'age_approx','anatom_site_general_challenge']]
preds = clf.predict_proba(test_df)[:,1]

#target = [round(value) for value in preds]


In [ ]:
#fetch data from CNN
df = pd.read_csv("../input/cnn-model-predictions/submission2.csv")
df =df["target"].values.tolist()

In [ ]:
els = []
for i in range(0,len(preds)):
    els.append(preds[i]*0.2+0.8*df[i])
sub["target"] = els
sub.to_csv('submission.csv', index=False)
sub.head()

In [ ]:
sub